QUANITATIVE MOMENTUM STRATEGY

Momentum investing means investing in stocks that have increaded in price the most

select 50 stocks with the heighest price Momentum. from there we recommend stocks with equal weight


In [ ]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

IMPORTING OUR LIST OF STOCKS

INIT API KEY

In [ ]:
# SMP 500 CSV
stocks = pd.read_csv("sp_500_stocks.csv")

# api
IEX_CLOUD_API_TOKEN = ''

SORTING ALL STOCKS INTO 2D ARRAY HOLDING A LIST OF 100 STOCKS EACH
[
    [100],
    [100],
    [100],
    [100],
    [100],
]

TURN EACH LIST INTO A STRING 
[
    'X' * 100 ,
    'X' * 100 ,
    'X' * 100 ,
    'X' * 100 ,
    'X' * 100 ,
]

In [ ]:
# function to split bigger list into smaller lists
def chunks( lst , n  ):
    ''' yeild successive n-sized lists from lst'''
    for i in range( 0 , len(lst) , n ):
        yield lst[i: i + n]

# 2 dimentional list with each inner list containing 100 tickers
symbol_groups = list(chunks( stocks['Ticker'] , 100  ) )
symbol_strings = []

# turns each list into usable string for api call
for i in range( 0 , len(symbol_groups) , 1  ):
    joined_arr =  ",".join(symbol_groups[i])
    symbol_strings.append( joined_arr )


CREATING FINAL OUTPUT PANDAS DATAFRAME

In [ ]:
# creating columns for pandas dataFrame  *** FOR LATER USER ***
my_columns = [  "Ticker"  , "Price" , "One-Year Price Return" , 'Numbers of Shares To Buy'   ]
final_dataframe = pd.DataFrame(  columns=my_columns   )

MAKING BATCH API CALL

In [ ]:
# create url for each list of 100 stocks
# request data for the sub list
for group_string in symbol_strings:
    batch_api_call = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={group_string}&types=advanced-stats,price&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get( batch_api_call ).json()

    # arr of every ticker in sub list
    stock_array_in_sub_list = group_string.split(',')
    for symbol in stock_array_in_sub_list:
        price = data[symbol]['price']
        oneYear_price_return = data[symbol]['advanced-stats']['year1ChangePercent']
        # append to final_dataframe
        final_dataframe = final_dataframe.append(  pd.Series( [symbol , price , oneYear_price_return , "N/A"] , index=my_columns )    ,   ignore_index=True  )


# final_dataframe

REMOVING LOW MOMENTUM STOCKS

"we are seeking to get the 50 highest momentum stocks in the SMP-500"

In [ ]:
# Sorting values in DECENDING order and saving it for later use using INPLACE = TRUE
final_dataframe.sort_values( "One-Year Price Return" , ascending = False , inplace = True  )

# save final_dataframe[:50] as original 
final_dataframe = final_dataframe[:50]

# Resetting index of sorted data_frame
final_dataframe.reset_index(  inplace = True  )

# # removing Old Index Column
final_dataframe.drop(labels='index', axis= 'columns', inplace=True, errors='raise')

final_dataframe

CALCULATING NUMBER OF SHARES TO BUY

In [ ]:
# validate user input for portfollio size '$$$'
def user_portfollio_size_input(portfollio_size):
    try:
        val = float(portfollio_size)
        print(f"Your have entered: ${val}\n")
        return True
    except ValueError:
        print("ERROR: Please type a numerical value")
        return False

portfollio_size = input("Please enter value of your portfollio: $")
while not user_portfollio_size_input(portfollio_size):
    portfollio_size = input("Please enter value of your portfollio: $")

In [ ]:
# calculate position size
position_size = float(portfollio_size) / len(final_dataframe.index)
print(f"Your position size is ${'{:,.2f}'.format(position_size)}")

In [ ]:
# update final data frame
for x in range(0 , len(final_dataframe) , 1):
    final_dataframe.loc[x , "Numbers of Shares To Buy"] =  position_size /  final_dataframe.loc[x , "Price"]


final_dataframe